### Python DOE 101
Python basics for DOE 

In [ ]:
%matplotlib widget
import numpy as np                  # numpy
import matplotlib.pyplot as plt     # matplotlib.pyplot
import pandas as pd                 # pandas
import statsmodels.api as sm        # statsmodels.api

from scipy.optimize import minimize                 # minimize
from sklearn.linear_model import LinearRegression   # LinearRegression
from statsmodels.formula.api import ols             # ols (Ordinary Least Squares)

In [ ]:
""" Generating data using numpy """
np.random.seed(42)  # Fix random seed 
# 2D line data
x = np.linspace(0, 1, num=5)
y0 = x + np.random.normal(0, 1, x.size)
z = x * y0

# 3D mesh data
X, Y = np.meshgrid(x, y0)
Z = 2*X + 0.5*Y

In [ ]:
""" Simple plot using matplotlib.pyplot """
fig = plt.figure(figsize=(8, 4))

# 2D line plot
ax2d1 = fig.add_subplot(2,2,1)
ax2d1.set_title('2D plot')
ax2d1.tick_params('x', labelbottom=False)
ax2d1.set_ylabel('$y$')
ax2d1.plot(x, y0)
ax2d1 = fig.add_subplot(2,2,3)
ax2d1.set_xlabel('$x$')
ax2d1.set_ylabel('$z$')
ax2d1.plot(x, z)

# 3D surface plot
ax3d = fig.add_subplot(2,2,(2,4),projection='3d')
ax3d.set_proj_type('ortho')
ax3d.view_init(30, -165)
ax3d.set_title('3D plot')
ax3d.set_xlabel('$x$')
ax3d.set_ylabel('$y$')
ax3d.set_zlabel('$z$')
ax3d.plot_surface(X, Y, Z, alpha=0.5)
ax3d.scatter(X, Y, Z, color='b', alpha=1)

plt.tight_layout()
plt.show()

In [ ]:
""" Experiment Design using numpy and pandas.DataFrame """
# Factor levels
A_levels = np.array([-1, 1])
B_levels = np.array([-1, 1])

# Number of levels per factor
n_A = len(A_levels)
n_B = len(B_levels)

# Total runs
n_runs = n_A * n_B

# Create the design matrix columns
A = np.repeat(A_levels, n_B)  # Repeat each level B times
B = np.tile(B_levels, n_A)    # Tile n_A times
print('A:', A)
print('B:', B)

# Stack columns into a design matrix
design = np.column_stack((A, B))
print('A B stack')
print(design)

# Convert to DataFrame
df = pd.DataFrame(design, columns=['A', 'B'])
display('Design as DataFrame')
display(df)

In [ ]:
# Replicate treatments (runs = 2*treatments)
df = pd.concat([df]*2, ignore_index=True)
display('Design as DataFrame after replicating')
display(df)

In [ ]:
# Adding response data
np.random.seed(42)
def measure_response(x1, x2):
    y0 =  0.0
    a  =  0.0
    b  =  0.0
    c  =  2.0
    y1 = y0 + a*x1 + b*x2 + c*x1*x2 + np.random.normal(0, 0.1, x1.shape)
    return y1
df['y'] = measure_response(df['A'], df['B'])
display(df)

In [ ]:
""" Model Fitting using statsmodels.OLS """
# Fit model with all interactions
model = ols('y ~ A + B + A:B', data=df).fit()
display(model.params)
_ = [print(f'B_{i}: {param:.3f}') for i, param in enumerate(model.params)]

In [ ]:
""" Analysis of Variance using statsmodels.stats.anova_lm (ANOVA for fitted linear models) """
# See https://www.r-bloggers.com/2011/03/anova-%e2%80%93-type-iiiiii-ss-explained/ for details on ANOVA Type 

# Perform an ANOVA (analysis of variance)
anova_table = sm.stats.anova_lm(model, typ=1)
display(anova_table)
anova_table = sm.stats.anova_lm(model, typ=2)
display(anova_table)
anova_table = sm.stats.anova_lm(model, typ=3)
display(anova_table)

In [ ]:
""" Optimising from Obtained Model """
def model_prediction(x):
    df = pd.DataFrame([x], columns=['A','B'])
    return model.predict(df)[0]

# Initial guess (centre point)
x0 = [0, 0]

# Bounds for each factor
bounds = [(-1, 1)] * 2
print(bounds)

# Minimise the predicted response
res_min = minimize(model_prediction, x0=x0, bounds=bounds)
print('Minimum predicted Y at:', res_min.x, 'Predicted Y:', res_min.fun)

# Maximise (by minimizing the negative)
res_max = minimize(lambda x: -model_prediction(x), x0=x0, bounds=bounds)
print('Maximum predicted Y at:', res_max.x, 'Predicted Y:', -res_max.fun)